# Create DWH

In [1]:
import sqlite3
from glob import glob
import pandas as pd
from bs4 import BeautifulSoup
import os
import requests

In [2]:
# Absoluten Pfad zum Projektverzeichnis und 'data-lake' Verzeichnis festlegen
PROJECT_DIR = os.path.dirname(os.getcwd())  # Eine Ebene nach oben vom 'scripts'-Ordner
DATA_LAKE_DIR = os.path.join(PROJECT_DIR, "data-lake")

In [3]:
def read_html_file(filename, encoding="utf-8"):
    full_path = os.path.join(PROJECT_DIR, filename)    
    try:
        # Versuch, die Datei mit der angegebenen Kodierung zu lesen
        with open(full_path, "r", encoding=encoding) as f:
            text = f.read()
    except UnicodeDecodeError:
        # Fallback auf ISO-8859-1, falls UTF-8 fehlschlägt
        #print(f"Warnung: Kodierung {encoding} fehlgeschlagen für {filename}, versuche iso-8859-1")
        with open(full_path, "r", encoding="iso-8859-1") as f:
            text = f.read()
    return text


In [4]:
file_name = os.path.join(DATA_LAKE_DIR, "2023-04-01-faz.html")
name = "faz"
date = "2022-04-01"
encoding = "utf-8"

In [5]:
stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(stopwords_url, allow_redirects=True).text.split("\n")[9:]
stopwords_list[0:20]

['ab',
 'aber',
 'abermaliges',
 'abermals',
 'abgerufen',
 'abgerufene',
 'abgerufener',
 'abgerufenes',
 'abgesehen',
 'acht',
 'aehnlich',
 'aehnliche',
 'aehnlichem',
 'aehnlichen',
 'aehnlicher',
 'aehnliches',
 'aehnlichste',
 'aehnlichstem',
 'aehnlichsten',
 'aehnlichster']

In [6]:
def parse_html(name, date, file_name, encoding):
    content = read_html_file(file_name, encoding)
    soup = BeautifulSoup(content, "html.parser")
    text = soup.text
    text = soup.get_text(separator=" ")
    items = text.replace("\n", " ").lower().split(" ")
    items = [i for i in items if len(i) > 1 and i not in stopwords_list]
    item_count = pd.Series(items).value_counts()
    count = item_count.to_frame()
    count.columns = ["count"]
    count["word"] = count.index
    count["paper"] = name
    count["date"] = date
    return count

In [7]:
count = parse_html(name, date, file_name, encoding)

In [8]:
count.sample(10)

,count,word,paper,date
blogs,3,blogs,faz,2022-04-01
subject,1,subject,faz,2022-04-01
krieges,1,krieges,faz,2022-04-01
außenpolitik:,1,außenpolitik:,faz,2022-04-01
straße.,1,straße.,faz,2022-04-01
wirtschaft,8,wirtschaft,faz,2022-04-01
do?,1,do?,faz,2022-04-01
paternoster-posern,1,paternoster-posern,faz,2022-04-01
bohrhammer,1,bohrhammer,faz,2022-04-01
cartoons,1,cartoons,faz,2022-04-01


## HTML-Dateien iterieren

In [9]:
# Strings zum Behalten
strings_to_keep = ['sz','zeit','faz', 'tagesspiegel', 'taz', 'abendblatt', 'berliner', 'welt', 'esslinger', 'handelsblatt', 'ntv', 'pioneer', 'suedwest', 'stuttgarter', 'dlf', 'dw-de', 'spiegel', 'mm', 'stern', 'tagesschau', 'wiwo', 'watson-de']

# Durchlaufen aller CSV-Dateien im Verzeichnis
for filename in os.listdir(DATA_LAKE_DIR):
    if filename.endswith(".csv"):
        file_path = os.path.join(DATA_LAKE_DIR, filename)

        # CSV-Datei mit Pandas einlesen
        df = pd.read_csv(file_path)

        # Behalten nur der Zeilen, die den angegebenen Strings in der Spalte "name" enthalten
        df = df[df['name'].isin(strings_to_keep)]

        # Speichern der bearbeiteten CSV-Datei
        df.to_csv(file_path, index=False)

In [10]:
def parse_csv_file(csv_file_name, sql_con=None):
    df = pd.read_csv(csv_file_name)
    if sql_con:
        df.to_sql("log", con=sql_con, if_exists="append")
    for i, row in df.iterrows():
        name = row["name"] 
        file_name = row["file_name"]
        date = row["date"]
        encoding = row["encoding"]
        count = parse_html(name, date, file_name, encoding)
        if sql_con:
            count.to_sql("count", con=sql_con, if_exists="append")
        #print(name, date)
        #print(count.shape)

In [11]:
csv_file_name = os.path.join(DATA_LAKE_DIR, "2023-04-01.csv")

In [12]:
parse_csv_file(csv_file_name)

## CSV-files iterieren

In [13]:
csv_filename_list = glob(os.path.join(DATA_LAKE_DIR, "*.csv"))

In [14]:
sql_con = sqlite3.connect(os.path.join(PROJECT_DIR, "db.sqlite3"))

In [15]:
for csv_file in csv_filename_list:
    df = pd.read_csv(csv_file)
    #print(csv_file)
    #print(len(df))
    parse_csv_file(csv_file, sql_con)

In [16]:
sql_con.close()